This notebook simulates stars at the center of the corner wavefront sensors, then uses `ts_wep` to estimate the zernikes.

In [ ]:
import os
import numpy as np
import yaml
from tqdm.notebook import tqdm
import galsim
import batoid
import wfsim
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from lsst.ts.wep.cwfs.Algorithm import Algorithm
from lsst.ts.wep.cwfs.CompensableImage import CompensableImage
from lsst.ts.wep.cwfs.Instrument import Instrument
from lsst.ts.wep.Utility import (
    CamType,
    DefocalType,
    getConfigDir,
    getModulePath
)

In [ ]:
rng = np.random.default_rng(5772156649015328606065120900824024310421)

In [ ]:
# set up the fiducial telescope
bandpass = galsim.Bandpass("LSST_r.dat", wave_type='nm')
fiducial_telescope = batoid.Optic.fromYaml("LSST_r.yaml")
factory = wfsim.SSTFactory(fiducial_telescope)
pixel_scale = 10e-6

In [ ]:
# Setup observation parameters.  Making ~plausible stuff up.
observation = {
    'zenith': 30 * galsim.degrees,
    'raw_seeing': 0.7 * galsim.arcsec,  # zenith 500nm seeing
    'wavelength': bandpass.effective_wavelength,
    'exptime': 15.0,  # seconds
    'temperature': 293.,  # Kelvin
    'pressure': 69.,  #kPa
    'H2O_pressure': 1.0  #kPa
}

In [ ]:
# Setup atmospheric parameters
atm_kwargs = {
    'screen_size': 819.2,
    'screen_scale': 0.1,
    'nproc': 6  # create screens in parallel using this many CPUs
}

In [ ]:
# perturb the mirrors
dof = rng.normal(scale=0.1, size=50) # activate some M2 bending modes
dof[[28, 45, 46]] = 0 # but zero-out the hexafoil modes that aren't currently fit well.
telescope = factory.get_telescope(dof=dof)  # no perturbations yet

In [ ]:
# now we simulate one (pair) of the corner wavefront sensors
# you set which pair by setting the raft.
# options are: R40 - R44
#               |     |
#              R00 - R04
raft = "R00"

# create the extrafocal simulator, i.e. the simulator for raft_SW0
extra = telescope.withGloballyShiftedOptic("Detector", [0, 0, +0.0015])
extra_simulator = wfsim.SimpleSimulator(
    observation,
    atm_kwargs,
    extra,
    bandpass,
    name=f"{raft}_SW0",
    rng=rng
)

# create the intrafocal simulator, i.e. the simulator for raft_SW1
intra = telescope.withGloballyShiftedOptic("Detector", [0, 0, -0.0015])
intra_simulator = wfsim.SimpleSimulator(
    observation,
    atm_kwargs,
    intra,
    bandpass,
    name=f"{raft}_SW1",
    rng=rng
)

In [ ]:
# set the star properties
star_T = 8000
sed = wfsim.BBSED(star_T)
flux = 10_000_000

In [ ]:
# simulate a star at the center of the extrafocal chip
extra_bounds = extra_simulator.get_bounds()
extra_thx = extra_bounds[0].mean()
extra_thy = extra_bounds[1].mean()
extra_simulator.add_star(extra_thx, extra_thy, sed, flux, rng)

# simulate a star at the center of the intrafocal chip
intra_bounds = intra_simulator.get_bounds()
intra_thx = intra_bounds[0].mean()
intra_thy = intra_bounds[1].mean()
intra_simulator.add_star(intra_thx, intra_thy, sed, flux, rng)

In [ ]:
# add the sky background
extra_simulator.add_background(1000.0, rng)
intra_simulator.add_background(1000.0, rng)

In [ ]:
# now we will plot the two chips

# first set up the figure and axes
# get horizontal/vertical orientation correct
if raft[1] == raft[2]:
    fix, axes = plt.subplots(ncols=1, nrows=2, figsize=(4, 4), sharex=True, sharey=True, dpi=120)
else:
    fix, axes = plt.subplots(ncols=2, nrows=1, figsize=(4, 4), sharex=True, sharey=True, dpi=120)
# make sure extrafocal chip is closer to center of focal plane
if raft[1] == "4":
    axes = axes[::-1]

# plot the extrafocal chip
axes[0].imshow(extra_simulator.image.array, origin="lower")
axes[0].text(
    0.02, 0.98, 
    f"{extra_simulator.sensor_name}\nextra", 
    transform=axes[0].transAxes, 
    ha="left", va="top", 
    c="w",
)

# plot the intrafocal chip
axes[1].imshow(intra_simulator.image.array, origin="lower")
axes[1].text(
    0.02, 0.98, 
    f"{intra_simulator.sensor_name}\nintra", 
    transform=axes[1].transAxes, 
    ha="left", va="top", 
    c="w",
)

plt.tight_layout()
plt.show()

In [ ]:
# now we will crop the donuts 

# first set up the figure and axes
# get horizontal/vertical orientation correct
if raft[1] == raft[2]:
    fix, axes = plt.subplots(ncols=1, nrows=2, figsize=(3, 6), sharex=True, sharey=True, dpi=120)
else:
    fix, axes = plt.subplots(ncols=2, nrows=1, figsize=(6, 3), sharex=True, sharey=True, dpi=120)
# make sure extrafocal chip is closer to center of focal plane
if raft[1] == "4":
    axes = axes[::-1]

# plot the extrafocal image
x, y = extra_simulator.wcs.radecToxy(extra_thx, extra_thy, galsim.radians) # donut center in x/y coords
x = int(x - extra_simulator.image.bounds.xmin) # x in image coords
y = int(y - extra_simulator.image.bounds.ymin) # y in image coords
extra_img = extra_simulator.image.array[y-128:y+128, x-128:x+128] # cut out the donut
axes[0].imshow(extra_img, origin="lower")
axes[0].text(
    0.02, 0.98, 
    f"{extra_simulator.sensor_name}\nextra", 
    transform=axes[0].transAxes, 
    ha="left", va="top", 
    c="w",
)


# plot the intrafocal image
x, y = intra_simulator.wcs.radecToxy(intra_thx, intra_thy, galsim.radians)
x = int(x - intra_simulator.image.bounds.xmin) # x in image coords
y = int(y - intra_simulator.image.bounds.ymin) # y in image coords
intra_img = intra_simulator.image.array[y-128:y+128, x-128:x+128] # cut out the donut
axes[1].imshow(intra_img, origin="lower")
axes[1].text(
    0.02, 0.98, 
    f"{intra_simulator.sensor_name}\nintra", 
    transform=axes[1].transAxes, 
    ha="left", va="top", 
    c="w",
)

plt.tight_layout()
plt.show()

Now we want to load the stuff from `ts_wep` to estimate the zernikes from the donuts

In [ ]:
# CWFS config
cwfsConfigDir = os.path.join(getConfigDir(), "cwfs")
instDir = os.path.join(cwfsConfigDir, "instData")
inst = Instrument(instDir)
algoDir = os.path.join(cwfsConfigDir, "algo")

In [ ]:
# run zernike estimation using the FFT algorithm
I_extra = CompensableImage()
I_extra.setImg(np.rad2deg([extra_thx, extra_thy]), DefocalType.Extra, image=extra_img.copy())

I_intra = CompensableImage()
I_intra.setImg(np.rad2deg([intra_thx, intra_thy]), DefocalType.Intra, image=intra_img.copy())
              
inst.config(CamType.LsstFamCam, I_extra.getImgSizeInPix(), announcedDefocalDisInMm=1.5)

fftAlgo = Algorithm(algoDir)
fftAlgo.config("fft", inst)          
fftAlgo.runIt(I_intra, I_extra, "offAxis", tol=1e-3)
fft_zk = fftAlgo.getZer4UpInNm()

# run zernike estimation using the Exp algorithm
# There's probably a reset method somewhere, but it's fast enough to just
# reconstruct these...
I_extra = CompensableImage()
I_extra.setImg(np.rad2deg([extra_thx, extra_thy]), DefocalType.Extra, image=extra_img.copy())

I_intra = CompensableImage()
I_intra.setImg(np.rad2deg([intra_thx, intra_thy]), DefocalType.Intra, image=intra_img.copy())
              
inst.config(CamType.LsstFamCam, I_extra.getImgSizeInPix(), announcedDefocalDisInMm=1.5)

expAlgo = Algorithm(algoDir)
expAlgo.config("exp", inst)          
expAlgo.runIt(I_intra, I_extra, "offAxis", tol=1e-3)
exp_zk = expAlgo.getZer4UpInNm()

In [ ]:
# get the true zernikes at the center of the corner wavefront chip
center_thx, center_thy = np.hstack((extra_bounds, intra_bounds)).mean(axis=1)
bzk = batoid.zernike(telescope, center_thx, center_thy, 622e-9, eps=0.61)
# convert wave -> nm
bzk *= 622

In [ ]:
# print all the zernikes
for i in range(4, 23):
    print(f"{i:2}  {exp_zk[i-4]:8.3f} nm  {fft_zk[i-4]:8.3f} nm  {bzk[i]:8.3f} nm")

fig, ax = plt.subplots(dpi=120)
ax.plot(range(4, 23), fft_zk, label='fft')
ax.plot(range(4, 23), exp_zk, label='exp')
ax.plot(range(4, 23), bzk[4:], label='truth')
plt.axhline(0, c='k')

ax.legend()

ax.set(xlabel="Noll index", ylabel="Perturbation amplitude (nm)")
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

plt.show()